In [2]:
import numpy as np
import gymnasium as gym
import rlenvs

# from scipy.special import softmax
# import pandas as pd

from rlcd.model import *

In [3]:
def discretize(s):
    cart_position, cart_velocity, pole_angle, angular_velocity = s
    
    def func(val, step=5, size=20, scale=.001):
        limits = [ i*scale for i in range(-(size-2)//2*step, (size+1)//2*step, step)]
        ids = [i for i,v in enumerate(limits) if val<v]
        return size-1 if len(ids) == 0 else ids[0]
    
    cart_velocity_params =  {"step": 2, "size": 20, "scale": .1}
    pole_angle_params =  {"step": 45, "size": 20, "scale": .0001}
    angular_velocity_params =  {"step": 192, "size": 4, "scale": .001}
    
    return (0, 
            func(cart_velocity, **cart_velocity_params),
            func(pole_angle, **pole_angle_params),
            func(angular_velocity, **angular_velocity_params)
    )


In [19]:
np.array(env.action_space)
# env.action_space

array(Discrete(2), dtype=object)

In [13]:
n=500

# env = gym.make("custom/CartPole-v1", render_mode="human")
env = gym.make("custom/CartPole-v1")
env.action_space.seed(82)

model = RLCD(np.array([s for s in range(2)]), np.array([0,1]))
agent = Dyna(model, n=100, alpha=.9, gamma=.9, epsilon=.1)

observation, info = env.reset(seed=82, masspole=.45, length=1.0)
hist_s = [discretize(observation)]
hist_a = []

/home/gui/Área de Trabalho/rlalgs/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/gui/Área de Trabalho/rlalgs/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


In [ ]:
for _ in range(n):
    # action = env.action_space.sample()
    action = agent.pi(hist_s[-1])
    # print('action: ', action)
    observation,reward, terminated, truncated, info = env.step(action)
    # print("observation : ",observation);
    
    hist_s.append(discretize(observation))
    hist_a.append(action)

    if terminated or truncated:
        break
        # observation, info = env.reset()
        
env.close()

In [112]:
# model = RLCD([0,1], [0], Emin=-0.000001)
model = RLCD([0,1], [0])

s = None
for s_ in data:
    if s is None:
        pass
    else:
        model.learn(s,0,s_,0, log=False)
    s = s_

model.sample(0,0)[0]

1

In [113]:
float(np.float64(3.0))

3.0

In [114]:
print("# p/ modelo: ", [m.N for m in model.Models])
print(
    "s=0 e s_=1 por modelo:",
    '#', [float(round(m.t.n[0,0,1],3)) for m in model.Models],
    'p=', [float(round(m.t.v[0,0,1],3)) for m in model.Models]

)

print(
    "s=1 e s_=1 por modelo:",
    '#', [float(round(m.t.n[1,0,1],3)) for m in model.Models],
    'p=', [float(round(m.t.v[1,0,1],3)) for m in model.Models]
)
print(
    "s=0 e s_=0 por modelo:",
    '#', [float(round(m.t.n[0,0,0],3)) for m in model.Models],
    'p=', [float(round(m.t.v[0,0,0],3)) for m in model.Models]
)

print(
    "s=1 e s_=0 por modelo:",
    '#', [float(round(m.t.n[1,0,0],3)) for m in model.Models],
    'p=', [float(round(m.t.v[1,0,0],3)) for m in model.Models]
)

# p/ modelo:  [906, 798, 295]
s=0 e s_=1 por modelo: # [0.0, 100.0, 0.0] p= [0.5, 0.999, 0.001]
s=1 e s_=1 por modelo: # [100.0, 0.0, 0.0] p= [1.0, 0.0, 0.5]
s=0 e s_=0 por modelo: # [0.0, 1.0, 100.0] p= [0.5, 0.001, 0.999]
s=1 e s_=0 por modelo: # [0.0, 100.0, 0.0] p= [0.0, 1.0, 0.5]


# Samples

In [115]:
print("# p/ modelo: ", [m.N for m in model.Models])

n = 1000
print(
    "Prob. s=0 e s_=1 por modelo:",
    [
        sum([m.sample(0,0)[0] for _ in range(n)])/n
        for m in model.Models
    ]
)

print(
    "Prob. s=1 e s_=1 por modelo:",
    [
        sum([m.sample(1,0)[0] for _ in range(n)])/n
        for m in model.Models
    ]
)
print(
    "Prob. s=0 e s_=0 por modelo:",
    [
        sum([1-m.sample(0,0)[0] for _ in range(n)])/n
        for m in model.Models
    ]
)

print(
    "Prob. s=1 e s_=0 por modelo:",
    [
        sum([1-m.sample(1,0)[0] for _ in range(n)])/n
        for m in model.Models
    ]
)

# p/ modelo:  [906, 798, 295]
Prob. s=0 e s_=1 por modelo: [0.509, 1.0, 0.001]
Prob. s=1 e s_=1 por modelo: [1.0, 0.0, 0.5]
Prob. s=0 e s_=0 por modelo: [0.513, 0.001, 0.999]
Prob. s=1 e s_=0 por modelo: [0.0, 0.999, 0.495]


In [116]:
n = 1000
print("Simulação s=0: ", sum([model.sample(0,0)[0] for _ in range(n)])/n)
print("Simulação s=1: ", sum([model.sample(1,0)[0] for _ in range(n)])/n)

Simulação s=0:  0.994
Simulação s=1:  0.994
